In [3]:
import requests
import pandas as pd
import re
from MyCreds.mycreds import Capstone_AWS_Stuart
from sqlalchemy import create_engine
import time
import geopandas as gpd
from geoalchemy2 import Geometry, Geography
import geocoder


engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_Stuart.username}:{Capstone_AWS_Stuart.password}@{Capstone_AWS_Stuart.host}/Capstone', echo=False)
sql_alc_cnxn = engine.connect()

In [4]:
locations = ['Central Business District',
             'Holland Village',
             'Dempsey Hill',
             'Serangoon Gardens Circus',
             'Orchard',
             'Arab Street',
             'Thomson Ridge',
             'East Coast Park',
             'Bishan Park',
             'Botanic Gardens',
             'Hort Park',
             'Sentosa',
             'Gardens By The Bay'
             ]


In [5]:

results = {
    'q_loc':[],
    'address':[],
    'confidence':[],
    'latitude':[],
    'longitude':[],
    'quality':[]
}

for location in locations:
    g = geocoder.arcgis(location+' Singapore')
    results['q_loc'].append(location.lower().replace(' ','_'))
    results['address'].append(g.json['address'])
    results['confidence'].append(g.json['confidence'])
    results['latitude'].append(g.json['lat'])
    results['longitude'].append(g.json['lng'])
    results['quality'].append(g.json['quality'])

df = pd.DataFrame(results)
df

,q_loc,address,confidence,latitude,longitude,quality
0,central_business_district,"Central Business District, Singapore",7,1.282720,103.851200,Locality
1,holland_village,Holland Village,7,1.311940,103.793330,Locality
2,dempsey_hill,DEMPSEY HILL,7,1.304750,103.809800,POI
3,serangoon_gardens_circus,"Serangoon Garden Circus, Singapore",9,1.363996,103.865944,StreetName
4,orchard,"Orchard, Singapore",7,1.301090,103.839650,Locality
5,arab_street,"Arab Street, Singapore",9,1.302327,103.858138,StreetName
6,thomson_ridge,"Thomson Ridge, 133 Thomson Ridge, 574715, Sing...",9,1.351379,103.833087,PointAddress
7,east_coast_park,East Coast Park,7,1.310920,103.949260,POI
8,bishan_park,Bishan Park I,7,1.364330,103.834990,POI
9,botanic_gardens,Singapore Botanic Gardens,7,1.315170,103.816380,POI


In [6]:
import folium

locations = df[['latitude','longitude']]
locationlist = locations.values.tolist()

map = folium.Map(location=[1.3521, 103.8198])
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df['address'][point]).add_to(map)
map

In [7]:
poi_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude,df.latitude))
poi_df.crs = "EPSG:4326"

In [8]:
poi_df

,q_loc,address,confidence,latitude,longitude,quality,geometry
0,central_business_district,"Central Business District, Singapore",7,1.282720,103.851200,Locality,POINT (103.85120 1.28272)
1,holland_village,Holland Village,7,1.311940,103.793330,Locality,POINT (103.79333 1.31194)
2,dempsey_hill,DEMPSEY HILL,7,1.304750,103.809800,POI,POINT (103.80980 1.30475)
3,serangoon_gardens_circus,"Serangoon Garden Circus, Singapore",9,1.363996,103.865944,StreetName,POINT (103.86594 1.36400)
4,orchard,"Orchard, Singapore",7,1.301090,103.839650,Locality,POINT (103.83965 1.30109)
5,arab_street,"Arab Street, Singapore",9,1.302327,103.858138,StreetName,POINT (103.85814 1.30233)
6,thomson_ridge,"Thomson Ridge, 133 Thomson Ridge, 574715, Sing...",9,1.351379,103.833087,PointAddress,POINT (103.83309 1.35138)
7,east_coast_park,East Coast Park,7,1.310920,103.949260,POI,POINT (103.94926 1.31092)
8,bishan_park,Bishan Park I,7,1.364330,103.834990,POI,POINT (103.83499 1.36433)
9,botanic_gardens,Singapore Botanic Gardens,7,1.315170,103.816380,POI,POINT (103.81638 1.31517)


In [9]:
poi_df.to_postgis('points_of_interests',engine, if_exists='replace', index=False, dtype={'geometry': Geometry(geometry_type='POINT', srid= 4326)})

In [10]:
distinct_list = poi_df.q_loc.tolist()

In [11]:
query_1 = "SELECT a.id, "
query_2 = "FROM sg_buildings_postal_geo a, "

for i in distinct_list:
    new_string_1 = 'dist_to_'+str(i)+', '
    query_1 = query_1+new_string_1
    new_string_2 = "LATERAL get_poi_dist('"+str(i)+"',a.geometry) "+ new_string_1
    query_2 = query_2+new_string_2

In [12]:
query_1+query_2

"SELECT a.id, dist_to_central_business_district, dist_to_holland_village, dist_to_dempsey_hill, dist_to_serangoon_gardens_circus, dist_to_orchard, dist_to_arab_street, dist_to_thomson_ridge, dist_to_east_coast_park, dist_to_bishan_park, dist_to_botanic_gardens, dist_to_hort_park, dist_to_sentosa, dist_to_gardens_by_the_bay, FROM sg_buildings_postal_geo a, LATERAL get_poi_dist('central_business_district',a.geometry) dist_to_central_business_district, LATERAL get_poi_dist('holland_village',a.geometry) dist_to_holland_village, LATERAL get_poi_dist('dempsey_hill',a.geometry) dist_to_dempsey_hill, LATERAL get_poi_dist('serangoon_gardens_circus',a.geometry) dist_to_serangoon_gardens_circus, LATERAL get_poi_dist('orchard',a.geometry) dist_to_orchard, LATERAL get_poi_dist('arab_street',a.geometry) dist_to_arab_street, LATERAL get_poi_dist('thomson_ridge',a.geometry) dist_to_thomson_ridge, LATERAL get_poi_dist('east_coast_park',a.geometry) dist_to_east_coast_park, LATERAL get_poi_dist('bishan_p